## Extraordinary Hackers. Gas Analysis

In [1]:
from config import PROJECT_ID, DATASET_NAME, INITIAL_TS, SNAPSHOT_TS, bq_client
from utils import drop_table, create_table

DROP_TABLES = True
CREATE_TABLES = True

table_name_1 = 'gas_spend_by_contract'
table_name_2 = 'gasdrop_distribution'

### Drop Tables

In [2]:
if DROP_TABLES:
    drop_table('gas_spend_by_contract')
    drop_table('gasdrop_distribution')

Table gasdrop:gas_spend_by_contract deleted.
Table gasdrop:gasdrop_distribution deleted.


### Create Table with Spending Gas by Contracts

In [3]:
query_1 = f'''
    CREATE TABLE `{PROJECT_ID}.{DATASET_NAME}.{table_name_1}` AS (
        WITH gas_spend AS (
            SELECT
                to_address,
                sum(cast(receipt_gas_used as numeric) * cast(gas_price as numeric))/1e18 as fee
            FROM `bigquery-public-data.crypto_ethereum.transactions`
            WHERE block_timestamp >= '{INITIAL_TS}'
              AND block_timestamp <= '{SNAPSHOT_TS}'
            GROUP BY to_address
            ORDER BY fee DESC
        ),
        traces AS (
            SELECT
                transaction_hash,
                from_address,
                to_address
            FROM `bigquery-public-data.crypto_ethereum.traces`
            WHERE block_timestamp >= '{INITIAL_TS}'
              AND block_timestamp <= '{SNAPSHOT_TS}'
              AND trace_type ='create'
              AND status = 1),
        contract_creators AS (
            SELECT
                from_address as creator_address,
                `hash` as transaction_hash
            FROM `bigquery-public-data.crypto_ethereum.transactions`
            WHERE block_timestamp >= '{INITIAL_TS}'
              AND block_timestamp <= '{SNAPSHOT_TS}')
        SELECT
            t.to_address as contract_address,
            if(t.from_address=cc.creator_address, null, t.from_address)  as fabric_address,
            cc.creator_address as creator_address,
            g.fee
        FROM gas_spend as g
        INNER JOIN traces as t ON g.to_address = t.to_address
        LEFT JOIN contract_creators as cc ON t.transaction_hash = cc.transaction_hash);
'''
if CREATE_TABLES:
    create_table(query_1, table_name_1)

Table gasdrop:gas_spend_by_contract created and filled.


### Create Table with Distribution by Contract and Fabric Creators

In [4]:
query_2 = f'''
    CREATE TABLE `{PROJECT_ID}.{DATASET_NAME}.{table_name_2}` AS (
        WITH fee_by_fabric AS (
            SELECT
                fabric_address,
                sum(fee) as sum_fee
            FROM `{PROJECT_ID}.{DATASET_NAME}.{table_name_1}`
            WHERE fabric_address is not null
            GROUP BY fabric_address
            ),
        fabric_creators AS (
            SELECT
                contract_address as fabric_address,
                creator_address as fabric_creator
            FROM `{PROJECT_ID}.{DATASET_NAME}.{table_name_1}`
            WHERE contract_address IN (
                SELECT DISTINCT fabric_address
                FROM `{PROJECT_ID}.{DATASET_NAME}.{table_name_1}`
                )
            )
        SELECT
            'fabric' as subtype,
            fabric_creator as address,
            fee_3 as sum_fee,
            CASE
                WHEN fee_3 > 10 THEN 3
                WHEN fee_3 > 0 THEN 2
                ELSE null
            END
            AS grade
        FROM (
            SELECT
                fabric_creator,
                sum(sum_fee) as fee_3
            FROM fee_by_fabric as ff
            LEFT JOIN fabric_creators as fc ON ff.fabric_address = fc.fabric_address
            GROUP BY fabric_creator)
        WHERE fabric_creator is not null
        UNION ALL
        SELECT
            'creator' as subtype,
            address,
            sum_fee,
            CASE
                WHEN sum_fee > 10 THEN 3
                WHEN sum_fee > 1 THEN 2
                WHEN sum_fee > 0 THEN 1
                ELSE null
            END
            AS grade
        FROM (
            SELECT
                creator_address as address,
                sum(fee) as sum_fee
            FROM `{PROJECT_ID}.{DATASET_NAME}.{table_name_1}`
            GROUP BY creator_address)
        WHERE address is not null
        ORDER BY sum_fee DESC);
'''

if CREATE_TABLES:
    create_table(query_2, table_name_2)

Table gasdrop:gasdrop_distribution created and filled.
